<a href="https://colab.research.google.com/github/vrra/FGAN-Build-a-thon/blob/main/Notebooks2023/Argilla_configure_dataset-v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created: 31 Dec 2023 (Happy New Year!!)

Aaron, Kennedy, Israel, Vishnu.

This notebook configures the dataset.

## Install Libraries

Install the latest version of Argilla in Colab, along with other libraries and models used in this notebook.

In [1]:
!pip install argilla datasets setfit evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

Prerequisites

Deploy Argilla Server on [HF Spaces](https://huggingface.co/new-space?template=argilla/argilla-template-space).


More info on Installation [here](../getting_started/installation/deployments/deployments.html).

## Secretes needed




* `ARGILLA_API_URL`: It is the url of the Argilla Server.
  * If you're using HF Spaces, it is constructed as `https://[your-owner-name]-[your_space_name].hf.space`.
* `ARGILLA_API_KEY`: It is the API key of the Argilla Server. It is `owner` by default.
* `HF_TOKEN`: It is the Hugging Face API token. It is only needed if you're using a [private HF Space](https://docs.argilla.io/en/latest/getting_started/installation/deployments/huggingface-spaces.html#deploy-argilla-on-spaces). You can configure it in your profile: [Setting > Access Tokens](https://huggingface.co/settings/tokens).
* `workspace`: admin


In [2]:
import argilla as rg
from argilla._constants import DEFAULT_API_KEY

In [3]:
from google.colab import userdata
api_url= userdata.get('my_argilla_url')
api_key= userdata.get('my_argilla_key')

import argilla as rg
rg.init(api_url=api_url, api_key=api_key)

# # If you want to use your private HF Space
# rg.init(extra_headers={"Authorization": f"Bearer {hf_token}"})

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:168: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


### Use a Task Template

Argilla offers a set of [pre-defined templates for different tasks](https://docs.argilla.io/en/latest/practical_guides/create_update_dataset/create_dataset.html#task-templates). You can use them to configure your dataset straightforward. For instance, if you want to create a dataset for simple text classification, you can use the following code:

In [4]:
dataset = rg.FeedbackDataset.for_supervised_fine_tuning(
    context=True,
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)
dataset


FeedbackDataset(
   fields=[TextField(name='prompt', title='Prompt', required=True, type='text', use_markdown=True), TextField(name='context', title='Context', required=False, type='text', use_markdown=True)]
   questions=[TextQuestion(name='response', title='Response', description='Write the response to the instruction.', required=True, type='text', use_markdown=True)]
   guidelines=This is a supervised fine-tuning dataset that contains instructions. Please write the response to the instruction in the response field. Take the context into account when writing the response.)
   metadata_properties=[])
)

In [9]:
dataset.field_by_name("prompt")

TextField(name='context', title='Context', required=False, type='text', use_markdown=True)

In [7]:
dataset.field_by_name("context")

TextField(name='context', title='Context', required=False, type='text', use_markdown=True)

In [8]:
dataset.question_by_name("response")

TextQuestion(name='response', title='Response', description='Write the response to the instruction.', required=True, type='text', use_markdown=True)

In [10]:
try:
    remote_dataset= dataset.push_to_argilla(name="my-first-dataset", workspace="admin")
except:
    pass

INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully pushed to Argilla
INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset(
   id=da9767dd-11d1-4772-b171-46537beef2b7
   name=my-first-dataset
   workspace=Workspace(id=4313bcb9-d5d2-4889-97c3-0cdcb4772830, name=admin, inserted_at=2023-12-30 16:09:58.983219, updated_at=2023-12-30 16:09:58.983219)
   url=https://vishnuramov-itu-t-build-a-thon.hf.space/dataset/da9767dd-11d1-4772-b171-46537beef2b7/annotation-mode
   fields=[RemoteTextField(id=UUID('dee99341-bf64-4735-8940-fb7cf22cde5d'), client=None, name='prompt', title='Prompt', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('8c49485c-6f8f-46b5-bda5-ebd0ef726b90'), client=None, name='context', title='Context', required=False, type='text', use_markdown=True)]
   questions=[RemoteTextQuestion(id=UUID('83ac683c-0fd6-4bfe-bf20-065579059833'), client=None, name='response', title='Response', description=None, required=True, type='t

-------------

